In [6]:
import pandas as pd
import geopandas as gpd
from pathlib import Path
import os
os.chdir("/Users/danielcp/Documents/2023_WORLD_BANK/21_CRP/12_New_Structure/city-scan-automation")
from config.paths import INPUTS, OUTPUTS
import yaml
from utils.aoi_module import find_country
from datetime import datetime as dt


In [2]:
from tasks.liquefaction import datacollection as liq_collect
from tasks.population_worldpop import datacollection as wp_collect

In [ ]:
city_inputs_path = INPUTS / "city_inputs.yml"
with open(city_inputs_path) as f:
    city_inputs = yaml.safe_load(f)
city_name = city_inputs['city_name'].replace(' ', '_').replace("'", "").lower()
# get country iso3 and country name
country_iso3, country_name = find_country(aoi=aoi)

# Update directories and make a copy of city inputs and menu in city-specific directory
if city_inputs.get('prev_run_date', None) is not None:
    cityscan_id = f"{city_inputs['prev_run_date']}-{country_name}-{city_name}"
else:
    cityscan_id = f"{dt.now().strftime('%Y-%m')}-{country_name}-{city_name}"

# generate output directory and folders
input_dir = OUTPUTS / f'{cityscan_id}/01-user-input'
output_dir = OUTPUTS / f'{cityscan_id}/02-process-output'
render_dir = OUTPUTS / f'{cityscan_id}/03-render-output'
os.makedirs(input_dir, exist_ok=True)
os.makedirs(f'{output_dir}/images/', exist_ok=True)
os.makedirs(f'{output_dir}/spatial/', exist_ok=True)
os.makedirs(f'{output_dir}/tabular/', exist_ok=True)
os.makedirs(render_dir, exist_ok=True)

# load AOI
aoi_path = INPUTS / f"AOI/{city_inputs['AOI_shp_name']}.shp"
aoi = gpd.read_file(aoi_path).to_crs(4326)

liq_array, liq_meta = liq_collect.datacollection(aoi=aoi, output_dir=output_dir)
